In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os
import sys
import cv2
sys.path.append('..')
import difflib
from fidelity_check import find_diff_writes

In [4]:
screenshot_similarity = json.load(open("../record_replay/screenshots/onload_screenshot_similarity.json", 'r'))
screenshot_similarity = {s['directory']: s['screenshot_similarity'] for s in screenshot_similarity for s in screenshot_similarity}

### Check the assumption of archive's writes are always a subset of live

### Fidelity check

In [5]:
dirr = '../revert_rewrite/writes/usps.com'
left = 'initial'
right = 'exception_SE'
find_diff_writes.fidelity_issue(dirr, left, right)


(True,
 ([['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[2]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[3]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[4]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[5]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[6]/div[2]/div[2]/table[1]/tbody[1]/tr[1]/td[1]/input[1]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[6]/div[2]/div[2]/table[1]/tbody[1]/tr[1]/td[2]/div[1]/div[1]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[6]/div[2]/div[2]/table[1]/tbody[1]/tr[1]/td[3]/input[1]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[9]']],
  [['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[2]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[3]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[4]'],
   ['/html[1]/body[1]/form[1]/div[1]/div[2]/div[1]/div[2]/div[5]'],
   ['/html[1

In [ ]:
dirs = os.listdir('../record_replay/writes')
num_diff, num_diff_writes = 0, 0
total = 0
for dirr in dirs:
    if not os.path.exists(f'../record_replay/writes/{dirr}/live_elements.json') or not os.path.exists(f'../record_replay/writes/{dirr}/archive_elements.json'):
        continue
    key_writes = find_diff_writes.locate_key_writes(f'../record_replay/writes/{dirr}')
    live_additional = [len(w['key_related_writes']) for w in key_writes['live']]
    archive_additional = [len(w['key_related_writes']) for w in key_writes['archive']]

    total += 1
    if len(live_additional) > 0 or len(archive_additional) > 0:
        num_diff += 1
    # if sum(archive_additional) > 0:
    #     num_diff_writes += 1

    ive_unique_elements, archive_unique_elements = find_diff_writes.find_diff_elements(f'../record_replay/writes/{dirr}')
    diff_writes = find_diff_writes.collect_diff_writes(f'../record_replay/writes/{dirr}')
    if len(diff_writes['archive']) > 0:
        print(len(diff_writes['archive']), len(archive_unique_elements))
        num_diff_writes += 1
print("Total", total)
print("Number of different layout trees", num_diff)
print("Number of archive's additional writes", num_diff_writes)

### fidelity check on a set of loads

In [14]:
def valid_dir(dirr):
    valid = os.path.exists(f'{dirr}/live_elements.json') and os.path.exists(f'{dirr}/archive_elements.json')
    return valid

base = '../record_replay/writes'
data = json.load(open(f'{base}/fidelity_check_metadata.json', 'r'))
results = []
for url, info in list(data.items()):
    dirr = info['directory']
    if not valid_dir(f"{base}/{dirr}"):
        continue
    issue, _ =  find_diff_writes.fidelity_issue(f"{base}/{dirr}")
    issue_screenshot, simi = find_diff_writes.fidelity_issue_screenshot(f"{base}/{dirr}")
    results.append({
        'directory': dirr,
        'issue': issue,
        'issue_screenshot': issue_screenshot,
        'screenshot_similarity': simi
    })
json.dump(results, open('fidelity_check.json', 'w+'), indent=2)